## 1)Загрузка данных о покупках и клиентах и объединение в один датафрейм

In [2]:
import pandas as pd
import numpy as np
import sqlite3

In [3]:
connection = sqlite3.connect('D:\\SKILLBOX\\DA Junior\\FINAL\\Data\\shop_database.db')
cursor = connection.cursor()

In [5]:
# Выгружаю из БД первую часть данных, с фильтром по стране
query = '''
    SELECT * 
    FROM personal_data
    WHERE country == 32;
'''
df_sql = pd.read_sql(query, connection)
df_sql

In [7]:
# Загружвю данные, которые потерялись
df_lost = pd.read_csv('D:\\SKILLBOX\\DA Junior\\FINAL\\Data\\personal_data.csv')
df_lost = df_lost[df_lost['country'] == 32]
df_lost

,id,age,education,city,country
0,101492,32,среднее,1188,32
1,42690,20,высшее,1134,32
2,113001,27,среднее,1187,32
3,163654,49,высшее,1160,32
4,69523,44,среднее,1137,32
...,...,...,...,...,...
15743,135183,44,среднее,1134,32
15744,125993,46,среднее,1134,32
15745,167019,71,среднее,1134,32
15746,116355,32,среднее,1188,32


In [8]:
# Объединяю данные в один датасет
df_personal_data = pd.concat([df_sql, df_lost], ignore_index=True)
df_personal_data = df_personal_data.sort_values(by='id')
df_personal_data

,id,gender,age,education,city,country
0,0,0.0,36,среднее,1201,32
91847,3,NaN,31,среднее,1134,32
1,4,0.0,35,среднее,1134,32
2,6,1.0,52,среднее,1188,32
3,7,0.0,37,среднее,1198,32
...,...,...,...,...,...,...
88782,177998,0.0,32,среднее,1201,32
88783,177999,1.0,34,среднее,1163,32
88784,178001,1.0,5,среднее,1208,32
92304,178002,NaN,50,среднее,1188,32


In [9]:
# Выгружаю из БД данные о покупках и персональные коэффициенты
query_1 = '''
    SELECT personal_data_coeffs.id, lbt_coef, ac_coef, sm_coef, personal_coef, 
           product, colour, cost, product_sex, base_sale, dt
    FROM purchases
    JOIN personal_data_coeffs ON purchases.id = personal_data_coeffs.id;
'''
df_sql_purchases = pd.read_sql(query_1, connection)
df_sql_purchases

,id,lbt_coef,ac_coef,sm_coef,personal_coef,product,colour,cost,product_sex,base_sale,dt
0,0,5.078678,-0.307147,0.959027,0.5072,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7
1,0,5.078678,-0.307147,0.959027,0.5072,Стол Outventure,зелёный,1499,NaN,0,37
2,0,5.078678,-0.307147,0.959027,0.5072,Набор Outventure: стол + 4 стула,бежевый,4799,NaN,0,37
3,3,7.764766,-0.030225,0.794720,0.4304,Бутсы мужские GSD Astro,белый,1599,1.0,0,13
4,3,7.764766,-0.030225,0.794720,0.4304,Мяч футбольный PUMA TEAMFINAL 21.2 FIFA QUALIT...,мультицвет,7199,NaN,0,27
...,...,...,...,...,...,...,...,...,...,...,...
786255,178004,8.437832,-0.080251,0.938592,0.4688,Полуботинки мужские Ascot Vortex,хаки/черный,7999,1.0,0,12
786256,178004,8.437832,-0.080251,0.938592,0.4688,Кроссовки мужские Nike Air Max 90,черный/серый,24299,1.0,0,13
786257,178004,8.437832,-0.080251,0.938592,0.4688,Кроссовки мужские Nike Air Max 90,черный/серый,24299,1.0,0,20
786258,178004,8.437832,-0.080251,0.938592,0.4688,Пакет,коричневый,12,NaN,0,20


In [10]:
# проверяю количество уникальных id в обеих таблицах
query_2 = '''
    SELECT 
    COUNT(DISTINCT personal_data_coeffs.id) as from_personal_data_coeffs,
    COUNT(DISTINCT purchases.id) as purchases
    FROM purchases
    JOIN personal_data_coeffs ON purchases.id = personal_data_coeffs.id;
'''
check = pd.read_sql(query_2, connection)
check

In [12]:
# Собираю все данные в один датасет
df_total = pd.merge(df_personal_data, df_sql_purchases, on='id', how='left')
df_total = df_total[df_total['country'] == 32]
df_total

,id,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef,product,colour,cost,product_sex,base_sale,dt
0,0,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7
1,0,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Стол Outventure,зелёный,1499,NaN,0,37
2,0,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Набор Outventure: стол + 4 стула,бежевый,4799,NaN,0,37
3,3,NaN,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304,Бутсы мужские GSD Astro,белый,1599,1.0,0,13
4,3,NaN,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304,Мяч футбольный PUMA TEAMFINAL 21.2 FIFA QUALIT...,мультицвет,7199,NaN,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780112,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Полуботинки мужские Ascot Vortex,хаки/черный,7999,1.0,0,12
780113,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Кроссовки мужские Nike Air Max 90,черный/серый,24299,1.0,0,13
780114,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Кроссовки мужские Nike Air Max 90,черный/серый,24299,1.0,0,20
780115,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Пакет,коричневый,12,NaN,0,20


In [13]:
# Промежуточное сохранение

#df_total = pd.read_csv('D:\SKILLBOX\DA Junior\FINAL\Data\df_total.csv')

## 2) Преобразование наименования товаров и цветов, заполнение product_sex

In [15]:
# Заполняю колонку 'product_sex' на основе колонки 'product'
# Если в 'product' есть слово 'мальчиков', то 3
# Если есть слово 'девочек', то 4
# Если есть 'детский' или 'юниорский', то 5, иначе 2(остальное)


df_total['product_sex'] = df_total['product_sex'].fillna(df_total['product'].apply(lambda x: 
                                                                                   3 if 'мальчиков' in x 
                                                                                   else (4 if 'девочек' in x 
                                                                                   else (5 if 'детский' in x or 'юниор' in x 
                                                                                   else 2))))
df_total

,id,gender,age,education,city,country,lbt_coef,ac_coef,sm_coef,personal_coef,product,colour,cost,product_sex,base_sale,dt
0,0,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,"Велосипед горный женский Stern Mira 2.0 26""",белый/синий,13599,0.0,1,7
1,0,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Стол Outventure,зелёный,1499,2.0,0,37
2,0,0.0,36,среднее,1201,32,5.078678,-0.307147,0.959027,0.5072,Набор Outventure: стол + 4 стула,бежевый,4799,2.0,0,37
3,3,NaN,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304,Бутсы мужские GSD Astro,белый,1599,1.0,0,13
4,3,NaN,31,среднее,1134,32,7.764766,-0.030225,0.794720,0.4304,Мяч футбольный PUMA TEAMFINAL 21.2 FIFA QUALIT...,мультицвет,7199,2.0,0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780112,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Полуботинки мужские Ascot Vortex,хаки/черный,7999,1.0,0,12
780113,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Кроссовки мужские Nike Air Max 90,черный/серый,24299,1.0,0,13
780114,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Кроссовки мужские Nike Air Max 90,черный/серый,24299,1.0,0,20
780115,178004,1.0,52,высшее,1188,32,8.437832,-0.080251,0.938592,0.4688,Пакет,коричневый,12,2.0,0,20


In [16]:
# Оставляю самый главный цвет(идущий первым), если ячейка не пустая
df_total['colour'] = df_total['colour'].apply(lambda x: x.lower().split('/')[0] if isinstance(x, str) else x)

In [17]:
# Оставляю цвет, идущий после тире, так как он является главным, так же обрабатываю варианты с точками
df_total['colour'] = df_total['colour'].apply(lambda x: x.split('-')[-1].strip() if isinstance(x, str) and '-' in x else 
                                  (x.split('.')[1] if isinstance(x, str) and '.' in x else x))

##### Далее цвета имеющие небольшое количество значений будут преобразованы в other

In [18]:
# Так как товаров очень много, преобразовываю их в группы
# Данныя функция позволяет избавиться от принадлежности к полу, так как есть отдельная колонка
import re
def process_product(row):
    product = row['product']
    sex = row['product_sex']
    
    if sex == 0.0:
        if 'женск' in product:
            product = product.split('женск')[0]
        elif 'женщин' in product:
            product = product.split('женщин')[0] 
    elif sex == 1.0:
        if 'мужск' in product:
            product = product.split('мужск')[0]
        elif 'мужчин' in product:
            product = product.split('мужчин')[0] 
    
    elif sex == 3.0:
        if 'мальчик' in product:
            product = product.split('мальчик')[0] 
    elif sex == 4.0:
        if 'девочек' in product:
            product = product.split('девочек')[0] 
    elif sex == 5.0:
        for word in product.split():
            if word in ['детский', 'детские', 'детское', 'детская','детей', 'юниорский', 'юниорское', 'юниорская']:
                product = ' '.join(product.split()[:product.split().index(word) ])
    elif sex == 2.0:
        for word in product.split():
             if word and re.match(r'^[a-zA-Z]', word[0]):
                product = ' '.join(product.split()[:product.split().index(word) ])
                break
    
    return product

In [19]:
# применяю функцию к датасету
df_total['product'] = df_total.apply(process_product, axis=1)

In [20]:
# Подсчет количества каждого значения в df_total['product']
product_counts = df_total['product'].value_counts()

# Преобразование значений в 'other', если их количество меньше 100, с удалением слова 'для' в конце
df_total['product'] = df_total['product'].apply(lambda x: 'other' if product_counts[x] < 100 else x.lower().rsplit(' для', 1)[0].strip())

In [22]:
# Функция для определения категории товара, так как товаров очень много, а четких указаний не было
categories = {
    'shoes': ['кроссовки', 'кеды', 'полуботинки', 'сандалии', 'шлепанцы', 'слипоны', 'ботинки', 'бутсы',
              'сабо', 'тапочки', 'стельки', 'туфли', 'боксеры', 'сапоги', 'чешки', 'балетки',
              'эспадрильи', 'шнурки'],
    'underwear': ['лиф', 'носки', 'топ', 'трусы', 'термобелье', 'гетры', 'гольфы', 'гамаши'],
    'wear': ['брюки', 'футболка', 'шорты', 'худи', 'толстовка', 'джемпер',
             'свитшот', 'лонгслив', 'легинсы', 'майка', 'поло', 'костюм',
             'велосипедки', 'рубашка', 'платье', 'рашгард', 'туника', 'бриджи',
             'тайтсы', 'юбка-шорты', 'шорты', 'олимпийка', 'комбинезон', 'худи', 'брюки'],
    'outerwear': ['ветровка', 'куртка', 'пуховик', 'жилет', 'перчатки', 'пальто', 'анорак', 'дождевик', 'накидка-дождевик', 'бомбер'],
    'bags': ['рюкзак', 'сумка', 'сумка-шоппе', 'рюкзаки', 'термосумка', 'рюкзак-жилет', 'накидка'],
    'hike': ['палатка', 'кресло', 'тент', 'спальный мешок', 'коврик', 'мешок', 'фляжка', 'бутылка', 'нож', 'гермомешок',
                'кухня', 'газовый', 'стул', 'кровать', 'фонарь', 'мультитул', 'несессер', 'следопыт', 'кемпинговая', 'плед', 'чайник',
                'аккумулятор', 'термоконтейнер', 'трос', 'вкладыш', 'набор посуды', 'кружка', 'стол', 'набор'],
    'headdress': ['бейсболка', 'панама', 'шапка', 'гейтор', 'кепка', 'повязка', 'бафф', 'козырек', 'бандана', 'бейсболки'],
    'bike': ['велосипед', 'самокат', 'электросамокат', 'шлем', 'флягодержатель', 'подножка', 'велошорты', 'седло', 'сидушка', 'покрышка',
                'звонок', 'круизер', 'компьютер', 'покрышка', 'замок', 'защита', 'велосипедных', 'велоботинки', 'крыло',
                'камера', 'ремонтный', 'мини-круизер', 'грипсы', 'багажник', 'электровелосипед', 'инструментов', 'вилка', 'лонгборд', 'звезд'],
    'swiming': ['купальник', 'плавки', 'плавательные', 'шапочка', 'надувной', 'бикини', 'ласты'],
    'accessory': ['очки', 'чехол', 'набор велосипедных', 'перчатки велосипедные', 'набор защиты', 
                  'полотенце', 'гантелей', 'кошелек', 'щитки', 'защиты'],
    'sports_goods': ['мяч', 'ракетка', 'коньки', 'коврик', 'палки', 'маска', 'лыжи', 'скейтборд', 
                     'стрессбол', 'эспандер', 'суппорт', 'суппорт', 'коньки', 'диск', 'гриф',
                     'скакалка', 'наколенники', 'гантель', 'гиря'],
    'Multigym': ['беговая дорожка', 'лента силовая', 'эллиптический', 'тренажер'],
    'wear': ['брюки', 'футболка', 'шорты', 'худи', 'толстовка', 'джемпер', 
             'свитшот', 'лонгслив', 'легинсы', 'майка', 'поло', 'костюм', 
             'велосипедки', 'рубашка', 'платье', 'рашгард', 'туника', 'бриджи']
} 
# Функция для определения категории товара
def categorize_product(product):
    for word in product.lower().split():
        for category, items in categories.items():
            if word in items:
                return category
    return 'other'  

# Создание нового столбца 'categories' на основе сравнения со значениями в категориях
df_total['categories'] = df_total['product'].apply(categorize_product)

In [23]:
pd.set_option('display.max_rows', None)
df_total['categories'].value_counts()

categories
shoes           249383
wear            214208
other            76839
outerwear        55625
hike             38967
bike             33346
bags             24752
underwear        24225
headdress        19366
sports_goods     16058
swiming          16041
accessory        10569
Multigym           738
Name: count, dtype: int64

In [ ]:
df_total.to_csv('D:\SKILLBOX\DA Junior\FINAL\Data\df_total.csv', index=False)

## 3) Чтение и преобразование данных для AB-теста

In [ ]:
import re
with open('D:\SKILLBOX\DA Junior\FINAL\Data\ids_first_company_negative.txt', 'r') as file:
    data = file.read()

values = data.split(';')
values = [val.strip() for val in values if val.strip()]

# Разделение каждого элемента списка по символу ","
separated_values = [item.split(',') for item in values]

# Применение метода strip() к каждому элементу во вложенных списках
separated_values_stripped = [[val.strip() for val in item] for item in separated_values]

# Создание нового пустого списка, в который будут собраны все элементы
flat_list = []

# Перебор элементов в исходном списке
for sublist in separated_values_stripped:
    for item in sublist:
        # Разделение элемента по пробелу и добавление разделенных значений в плоский список
        items = item.split(' ')
        flat_list.extend(items)
#очистка списка от слов и лишних символов
filtered_list = [re.sub("[^0-9]", "", x) for x in flat_list]

# Отфильтрованный список с числовыми значениями
filtered_list_negative = list(filter(None, filtered_list))  # Удаление пустых значений
print(filtered_list_negative)


In [ ]:
with open('D:\SKILLBOX\DA Junior\FINAL\Data\ids_first_company_positive.txt', 'r') as file:
    data = file.read()

# Разделение, очистка и объединение значений в один список
filtered_list_positive = re.findall(r'\d+', data)

print(filtered_list_positive)

In [ ]:
sorted_list_negative = sorted(filtered_list_negative) 
sorted_list_positive = sorted(filtered_list_positive) 


In [ ]:
df_negative = pd.DataFrame(sorted_list_negative, columns=['id'])
df_positive = pd.DataFrame(sorted_list_positive, columns=['id'])

In [ ]:
# Сохраняю списки id для AB-теста
df_negative.to_csv('D:\SKILLBOX\DA Junior\FINAL\Data\df_negative.csv', index=False)
df_positive.to_csv('D:\SKILLBOX\DA Junior\FINAL\Data\df_positive.csv', index=False)